In [1]:
import pandas as pd
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
import re
import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

## Reading the Dataset

In [2]:
data = pd.read_csv("./spam.csv", encoding='latin-1').sample(frac=1).drop_duplicates()
data = data[['v1', 'v2']].rename(columns={"v1":"label", "v2":"text"})
data.head()

,label,text
4044,ham,If You mean the website. Yes.
3849,ham,I to am looking forward to all the sex cuddlin...
1714,ham,Yeah I don't see why not
2241,ham,U buy newspapers already?
3582,ham,I sent your maga that money yesterday oh.


### Data Cleaning
#### Remove the Punctuations, stopwords, Lemmatize

In [3]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    return text

data['cleaned_text'] = data['text'].apply(lambda x: clean_text(x))
data.head()

,label,text,cleaned_text
4044,ham,If You mean the website. Yes.,mean websit ye
3849,ham,I to am looking forward to all the sex cuddlin...,look forward sex cuddl two sleep
1714,ham,Yeah I don't see why not,yeah dont see
2241,ham,U buy newspapers already?,u buy newspap alreadi
3582,ham,I sent your maga that money yesterday oh.,sent maga money yesterday oh


In [4]:
#data=data.drop(["text"],axis=1)
#data.head()

## Test and Train Dataset Split

In [5]:
data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

## Use BERT Embeddings and Train the model

In [6]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
#word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=100)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.


2020-02-05 12:12:34,689 Reading data from .
2020-02-05 12:12:34,690 Train: train.csv
2020-02-05 12:12:34,691 Dev: dev.csv
2020-02-05 12:12:34,692 Test: test.csv


C:\ProgramData\Anaconda3\lib\site-packages\flair\data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
C:\ProgramData\Anaconda3\lib\site-packages\flair\data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
C:\ProgramData\Anaconda3\lib\site-packages\flair\data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is mo

2020-02-05 12:12:44,526 Computing label dictionary. Progress:


100%|██████████████████████████████████████████████████████████████████████████| 4135/4135 [00:00<00:00, 318123.32it/s]


...

2020-02-05 13:43:16,851 epoch 37 - iter 91/130 - loss 0.00099564 - samples/sec: 186.64
2020-02-05 13:43:19,981 epoch 37 - iter 104/130 - loss 0.00102099 - samples/sec: 201.95
2020-02-05 13:43:23,145 epoch 37 - iter 117/130 - loss 0.00101857 - samples/sec: 195.22
2020-02-05 13:43:26,525 ----------------------------------------------------------------------------------------------------
2020-02-05 13:43:26,527 EPOCH 37 done: loss 0.0011 - lr 0.0008
2020-02-05 13:43:28,947 DEV : loss 0.04785548523068428 - score 0.9923
2020-02-05 13:43:29,000 BAD EPOCHS (no improvement): 2
2020-02-05 13:43:30,568 ----------------------------------------------------------------------------------------------------
2020-02-05 13:43:30,701 epoch 38 - iter 0/130 - loss 0.00005391 - samples/sec: 3200.84
2020-02-05 13:43:35,394 epoch 38 - iter 13/130 - loss 0.00135510 - samples/sec: 187.34
2020-02-05 13:43:39,010 epoch 38 - iter 26/130 - loss 0.00135449 - samples/sec: 158.87
2020-02-05 13:43:44,267 epoch 38 - ite

2020-02-05 13:47:13,609 EPOCH 43 done: loss 0.0010 - lr 0.0004
2020-02-05 13:47:16,016 DEV : loss 0.047963835299015045 - score 0.9923
Epoch    42: reducing learning rate of group 0 to 1.9531e-04.
2020-02-05 13:47:16,073 BAD EPOCHS (no improvement): 4
2020-02-05 13:47:17,453 ----------------------------------------------------------------------------------------------------
2020-02-05 13:47:17,641 epoch 44 - iter 0/130 - loss 0.00218877 - samples/sec: 2248.75
2020-02-05 13:47:22,526 epoch 44 - iter 13/130 - loss 0.00077225 - samples/sec: 186.47
2020-02-05 13:47:25,637 epoch 44 - iter 26/130 - loss 0.00096878 - samples/sec: 194.49
2020-02-05 13:47:30,758 epoch 44 - iter 39/130 - loss 0.00115960 - samples/sec: 199.34
2020-02-05 13:47:33,506 epoch 44 - iter 52/130 - loss 0.00108847 - samples/sec: 234.38
2020-02-05 13:47:36,872 epoch 44 - iter 65/130 - loss 0.00106321 - samples/sec: 208.95
2020-02-05 13:47:40,085 epoch 44 - iter 78/130 - loss 0.00107350 - samples/sec: 190.66
2020-02-05 13:4

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:574: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


2020-02-05 13:56:26,060 0.9787	0.9787	0.9787
2020-02-05 13:56:26,062 
MICRO_AVG: acc 0.9583 - f1-score 0.9787
MACRO_AVG: acc 0.9047 - f1-score 0.94855
ham        tp: 451 - fp: 8 - fn: 3 - tn: 55 - precision: 0.9826 - recall: 0.9934 - accuracy: 0.9762 - f1-score: 0.9880
spam       tp: 55 - fp: 3 - fn: 8 - tn: 451 - precision: 0.9483 - recall: 0.8730 - accuracy: 0.8333 - f1-score: 0.9091
2020-02-05 13:56:26,063 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9787,
 'dev_score_history': [0.9787,
  0.9884,
  0.9903,
  0.9865,
  0.9903,
  0.9884,
  0.9903,
  0.9903,
  0.9903,
  0.9903,
  0.9923,
  0.9923,
  0.9903,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923,
  0.9923],
 'train_loss_history': [0.14099423999163824,
  0.06762746933829755,
  0.04583877539620376,
  0.04146887439486678,
  0.03401835021885255,
  0.029118352060994276,
  0.022270211470850673,
  0.013809197267087606,
  0.011601482939659036,
  0.008063341164961458,
  0.007215783825421778,
  0.005273395067510697,
  0.003954336989241151,
  0.0041253711638721425,
  0.0028007527158339283,
  0.0018558785946180042,
  0.001759871122025148,
  0.002248157912883751,
  0.0012722291891875033,
  0.0016